# Preparing the data

In [34]:
import numpy as np

In [35]:
#Dataset
train1 = np.array([[2, 0, 0, 0, "Уворачиваться"],
                [2,0 , 0 ,1, "Уворачиваться"],
                [2, 0, 1, 1,"Атаковать"],
                [2, 0, 1, 2, "Атаковать"],
                [2, 1, 0, 2, "Прятаться"],
                [2, 1, 0, 1, "Атаковать"],
                [1, 0, 0, 0, "Уворачиваться"],
                [1, 0, 0, 1, "Прятаться"],
                [1, 0, 1, 1, "Атаковать"],
                [1, 0, 1, 2, "Прятаться"],
                [1, 1, 0, 2, "Прятаться"],
                [1, 1, 0, 1, "Прятаться"],
                [0, 0, 0, 0, "Уворачиваться"],
                [0, 0, 0, 1, "Прятаться"],
                [0, 0, 1, 1, "Прятаться"],
                [0, 0, 1, 2, "Бежать"],
                [0, 1, 0, 2, "Бежать"],
                [0, 1, 0, 1, "Прятаться"]])

test1 = np.array([[2, 1, 1, 1, "Атаковать"],
                    [1, 1, 1, 2, "Прятаться"],
                    [0, 0, 0, 0, "Уворачиваться"],
                    [0, 1, 1, 1, "Прятаться"],
                    [2, 0, 1, 3, "Прятаться"],
                    [2, 1, 0,3, "Прятаться"],
                    [2, 0, 1, 3, "Бежать"]])

In [36]:
y_train1 = train1[:, -1]
y_test1 = test1[:, -1]
X_train1 = train1[:, [0, 1, 2, 3]]
X_test1 = test1[:, [0, 1, 2, 3]]

# Models and predictions

## DecisionTree

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [38]:
DT = DecisionTreeClassifier(class_weight = "balanced", criterion = "entropy")
DT.fit(X_train1, y_train1)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [39]:
y_pred_DT = DT.predict(X_test1)

In [40]:
scoreDT = metrics.accuracy_score(y_test1, y_pred_DT)
scoreDT

0.7142857142857143

## Pytorch neural network

In [41]:
import torch

In [42]:
#Dataset with coded categorical features
# Уворачиваться= 1, Атаковать = 2, Прятаться = 3, Бежать = 0
train2 = np.array([[2, 0, 0, 0, 1],
                [2,0 , 0 ,1, 1],
                [2, 0, 1, 1,2],
                [2, 0, 1, 2, 2],
                [2, 1, 0, 2, 3],
                [2, 1, 0, 1, 2],
                [1, 0, 0, 0, 1],
                [1, 0, 0, 1, 3],
                [1, 0, 1, 1, 2],
                [1, 0, 1, 2, 3],
                [1, 1, 0, 2, 3],
                [1, 1, 0, 1, 3],
                [0, 0, 0, 0, 1],
                [0, 0, 0, 1, 3],
                [0, 0, 1, 1, 3],
                [0, 0, 1, 2, 0],
                [0, 1, 0, 2, 0],
                [0, 1, 0, 1, 3]])

test2 = np.array([[2, 1, 1, 1, 2],
                    [1, 1, 1, 2, 3],
                    [0, 0, 0, 0, 1],
                    [0, 1, 1, 1, 3],
                    [2, 0, 1, 3, 3],
                    [2, 1, 0,3, 3],
                    [2, 0, 1, 3, 0]])

In [43]:
y_train2 = train2[:, -1]
y_test2 = test2[:, -1]
X_train2 = train2[:, [0, 1, 2, 3]]
X_test2 = test2[:, [0, 1, 2, 3]]

In [45]:
# Transform data into TorchTensor
X_train2 = torch.from_numpy(X_train2).float()
X_test2 = torch.from_numpy(X_test2).float()
y_train2 = torch.from_numpy(y_train2)
y_test2 = torch.from_numpy(y_test2)

In [67]:
#Init neural network class
class NN(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(NN, self).__init__()
        
        self.fc1 = torch.nn.Linear(4, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 4)
        self.sm = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    

In [68]:
#NN with 3 neurons in hidden layer
NN = NN(3)

In [69]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(NN.parameters(), 
                             lr=1.0e-3)

In [70]:
batch_size = 3

for epoch in range(1000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = NN.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = NN.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor(0.2857)
tensor(0.5714)
tensor(0.5714)
tensor(0.5714)
tensor(0.5714)
tensor(0.5714)
tensor(0.7143)
tensor(0.8571)
tensor(0.8571)
tensor(0.8571)
